# 课程目标：在python环境下对临床资料进行整理

## 步骤
    1.读取：HIS和LIS数据
    2.整理：
    3.数据描述
    4.统计分析
    5.bonus：机器学习

In [1]:
#准备工作：导入模块/软件库

import numpy as np # 主要用于科学计算，可生成homogeneous数据，本例中用其生成易于展示图形，统计分析的数据
import pandas as pd # 用于导入数据，这些数据多为heterogeneous，是我们日常数据的主要形式
import matplotlib #matplotlib用于作图
import matplotlib.pyplot as plt
import seaborn as sns #seaborn用于高级制图，尤其是统计学用到的
from scipy import stats # 用于统计分析
import statsmodels.api as sm # 用于统计分析
import statsmodels.formula.api as smf # 用于统计分析
from datetime import datetime, date, time, timedelta #用于时间日期相关

In [2]:
# 设置显示参数，matplotlib 统一使用inline
%matplotlib inline

In [3]:
# pandas display option，调整显示r*w数
pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 20)

# 1. 读取并查看数据
    1.1 HIS数据，神经内科2013年12月至2019年6月份出院患者病历首页部分数据
    1.2 LIS数据，神经内科部分腰穿脑脊液数据（常规，生化部分项目）
            NS:患者姓名，身份证等信息已被移除
    1.3 使用info(),head(),tail()等查看数据

In [4]:
# when missing=False, you got NaN
# HIS data
discharge = pd.read_excel('/Users/yqzhang/Desktop/Meeting/20190808_CGME_Python/Neuro_Discharged.xls', 
                    convert_missing=False) 
# LIS data
csf = pd.read_csv('/Users/yqzhang/Desktop/Meeting/20190808_CGME_Python/csf.csv') 

In [ ]:
# 备用方案01 window，r'C:\users etc
# 网络存放数据库 'http://www.symphome.cn/y/Neuro_Discharged.xls' , 'http://www.symphome.cn/y/csf.csv'

In [14]:
discharge.head()

,出院科室,住院号,年龄,患者来源,诊断类型,诊断编码,诊断名称,入院日期,出院日期,住院天数,非药费,药费
0,神经内科,ZY010000007045,87,门诊,主要诊断,H81.000,梅尼埃病,2014-05-19 10:03:56,2000-05-27 10:08:37,0,3025.56,3111.17
1,神经内科,ZY010000007045,87,门诊,其他诊断,I10.x00,高血压病,2014-05-19 10:03:56,2000-05-27 10:08:37,0,3025.56,3111.17
2,神经内科,ZY010000007045,87,门诊,其他诊断,I25.103,冠状动脉粥样硬化性心脏病,2014-05-19 10:03:56,2000-05-27 10:08:37,0,3025.56,3111.17
3,神经内科,ZY010000033205,77,门诊,主要诊断,I61.900,脑出血,2015-06-28 13:26:04,2000-07-04 08:00:35,0,4033.59,3135.88
4,神经内科,ZY010000033205,77,门诊,其他诊断,E78.500,高脂血症,2015-06-28 13:26:04,2000-07-04 08:00:35,0,4033.59,3135.88


# 2. 整理数据(will occupy your 80% workload)
    以discharge数据为例，csf is available for analysis(wide table)
    2.1 删除不需要的observations(rows), variables(columns)
    2.2 修改variable name（字母简写）
    2.3 分裂数据
    2.4 合并数据（两表merge）

## 2.1 drop observations(rows) or variables(columns)

In [18]:
discharge.head()

,出院科室,住院号,年龄,患者来源,诊断类型,诊断编码,诊断名称,入院日期,出院日期,住院天数,非药费,药费
0,神经内科,ZY010000007045,87,门诊,主要诊断,H81.000,梅尼埃病,2014-05-19 10:03:56,2000-05-27 10:08:37,0,3025.56,3111.17
1,神经内科,ZY010000007045,87,门诊,其他诊断,I10.x00,高血压病,2014-05-19 10:03:56,2000-05-27 10:08:37,0,3025.56,3111.17
2,神经内科,ZY010000007045,87,门诊,其他诊断,I25.103,冠状动脉粥样硬化性心脏病,2014-05-19 10:03:56,2000-05-27 10:08:37,0,3025.56,3111.17
3,神经内科,ZY010000033205,77,门诊,主要诊断,I61.900,脑出血,2015-06-28 13:26:04,2000-07-04 08:00:35,0,4033.59,3135.88
4,神经内科,ZY010000033205,77,门诊,其他诊断,E78.500,高脂血症,2015-06-28 13:26:04,2000-07-04 08:00:35,0,4033.59,3135.88


In [19]:
# 删除变量’出院科室‘
discharge.drop(columns='出院科室', inplace=True)

In [20]:
discharge.head()

,住院号,年龄,患者来源,诊断类型,诊断编码,诊断名称,入院日期,出院日期,住院天数,非药费,药费
0,ZY010000007045,87,门诊,主要诊断,H81.000,梅尼埃病,2014-05-19 10:03:56,2000-05-27 10:08:37,0,3025.56,3111.17
1,ZY010000007045,87,门诊,其他诊断,I10.x00,高血压病,2014-05-19 10:03:56,2000-05-27 10:08:37,0,3025.56,3111.17
2,ZY010000007045,87,门诊,其他诊断,I25.103,冠状动脉粥样硬化性心脏病,2014-05-19 10:03:56,2000-05-27 10:08:37,0,3025.56,3111.17
3,ZY010000033205,77,门诊,主要诊断,I61.900,脑出血,2015-06-28 13:26:04,2000-07-04 08:00:35,0,4033.59,3135.88
4,ZY010000033205,77,门诊,其他诊断,E78.500,高脂血症,2015-06-28 13:26:04,2000-07-04 08:00:35,0,4033.59,3135.88


In [21]:
## 修改变量名，单个修改：df.rename(columns = {'sex':'Sex'})
discharge.columns=['IPD_ID', 'age', 'adm_site', 'diag_type', 'ICD',
             'diagnosis', 'adm_DT', 'dis_DT', 'stay_days', 'nondrug_fee', 'drug_fee']

In [22]:
discharge.head()

,IPD_ID,age,adm_site,diag_type,ICD,diagnosis,adm_DT,dis_DT,stay_days,nondrug_fee,drug_fee
0,ZY010000007045,87,门诊,主要诊断,H81.000,梅尼埃病,2014-05-19 10:03:56,2000-05-27 10:08:37,0,3025.56,3111.17
1,ZY010000007045,87,门诊,其他诊断,I10.x00,高血压病,2014-05-19 10:03:56,2000-05-27 10:08:37,0,3025.56,3111.17
2,ZY010000007045,87,门诊,其他诊断,I25.103,冠状动脉粥样硬化性心脏病,2014-05-19 10:03:56,2000-05-27 10:08:37,0,3025.56,3111.17
3,ZY010000033205,77,门诊,主要诊断,I61.900,脑出血,2015-06-28 13:26:04,2000-07-04 08:00:35,0,4033.59,3135.88
4,ZY010000033205,77,门诊,其他诊断,E78.500,高脂血症,2015-06-28 13:26:04,2000-07-04 08:00:35,0,4033.59,3135.88


## 2.3 分成两个表格，一个服务于我们查看卫生经济学，一个服务于临床诊断

In [23]:
# 卫生经济学表格, drop columns
dis_he = discharge[discharge.diag_type=='主要诊断'].reset_index(drop=True)

In [24]:
dis_he.tail()

,IPD_ID,age,adm_site,diag_type,ICD,diagnosis,adm_DT,dis_DT,stay_days,nondrug_fee,drug_fee
12107,ZY010000145350,70,门诊,主要诊断,G25.000,特发性震颤,2019-06-28 08:35:46,2019-07-08 07:48:13,10,7309.48,396.93
12108,ZY010000145252,56,门诊,主要诊断,I77.102,左侧锁骨下动脉狭窄,2019-06-27 08:49:38,2019-07-08 08:00:00,11,36746.90,2901.82
12109,ZY010000145488,47,急诊,主要诊断,I63.900,脑梗死,2019-06-29 14:03:03,2019-07-08 08:00:00,9,9588.06,3324.78
12110,ZY010000145923,56,门诊,主要诊断,G44.800,头痛综合征，其他特指的,2019-07-04 08:34:17,2019-07-08 10:18:59,4,5320.60,169.64
12111,ZY010000145468,38,急诊,主要诊断,A86.x00,病毒性脑炎,2019-06-29 10:10:31,2019-07-08 10:29:50,9,6038.82,1249.88


In [25]:
# 讲 adm_site 进行划分
dis_he['adm_site'] = dis_he['adm_site'].apply({'门诊':'OPD', '其他':'OPD', '其他医疗机构转入':'OPD', '急诊':'ER'}.get)

In [26]:
# drop observation if stay_days>30
dis_he = dis_he[dis_he.stay_days < 30]

In [27]:
# 临床诊断表格, drop variables
dis_cd = discharge.drop(columns=['adm_site', 'age', 'adm_DT', 'dis_DT', 'stay_days', 'nondrug_fee', 'drug_fee'])

In [28]:
dis_cd = dis_cd[dis_cd.ICD.isna()==False]

In [29]:
dis_cd.set_index(['IPD_ID', 'diag_type']).head()

ICD     diagnosis
IPD_ID         diag_type                       
ZY010000007045 主要诊断       H81.000          梅尼埃病
               其他诊断       I10.x00          高血压病
               其他诊断       I25.103  冠状动脉粥样硬化性心脏病
ZY010000033205 主要诊断       I61.900           脑出血
               其他诊断       E78.500          高脂血症

### 2.4 将临床诊断表格与脑脊液表格 合并

In [30]:
csf.head()

,IPD_ID,check_time,csf_wbc,csf_rbc,csf_glu,csf_tp,csf_cl
0,ZY010000038183,20150917,NaN,NaN,4.17,42.2,128.52
1,ZY010000061218,20160917,NaN,NaN,6.70,47.3,111.93
2,ZY010000075286,20170330,NaN,NaN,3.20,69.2,116.93
3,ZY010000081509,20170616,NaN,NaN,3.70,47.9,119.18
4,ZY010000071329,20170215,NaN,NaN,3.19,35.8,118.25


In [31]:
dis_cd.head(2)

,IPD_ID,diag_type,ICD,diagnosis
0,ZY010000007045,主要诊断,H81.000,梅尼埃病
1,ZY010000007045,其他诊断,I10.x00,高血压病


In [32]:
# merge tables of dis_cd & csf
csf_cd = csf.merge(dis_cd, on='IPD_ID', how='inner')

In [33]:
csf_cd.head()

,IPD_ID,check_time,csf_wbc,csf_rbc,csf_glu,csf_tp,csf_cl,diag_type,ICD,diagnosis
0,ZY010000038183,20150917,NaN,NaN,4.17,42.2,128.52,主要诊断,G61.801,慢性炎症性脱髓鞘性多发性神经病
1,ZY010000061218,20160917,NaN,NaN,6.70,47.3,111.93,主要诊断,A86.x00,病毒性脑炎
2,ZY010000061218,20160917,NaN,NaN,6.70,47.3,111.93,其他诊断,E11.900,2型糖尿病
3,ZY010000061218,20160917,NaN,NaN,6.70,47.3,111.93,其他诊断,F32.900,抑郁症
4,ZY010000061218,20160917,NaN,NaN,6.70,47.3,111.93,其他诊断,I25.103,冠状动脉粥样硬化性心脏病


In [35]:
# 显示可能为 炎性周围神经病的病例
csf_cd[csf_cd.ICD.str.contains('G6')].head(6)

,IPD_ID,check_time,csf_wbc,csf_rbc,csf_glu,csf_tp,csf_cl,diag_type,ICD,diagnosis
0,ZY010000038183,20150917,NaN,NaN,4.17,42.2,128.52,主要诊断,G61.801,慢性炎症性脱髓鞘性多发性神经病
31,ZY010000044484,20151231,1.0,0.0,3.33,28.7,125.95,主要诊断,G61.000,吉兰-巴雷综合征
35,ZY010000018917,20141111,NaN,NaN,3.79,17.3,126.78,主要诊断,G62.901,周围神经病
74,ZY010000086223,20170815,0.0,0.0,4.44,37.0,125.50,主要诊断,G61.000,慢性格林-巴利综合征
75,ZY010000086223,20170815,0.0,0.0,4.44,37.0,125.50,其他诊断,G62.802,多灶性运动神经病不排除
81,ZY020000054995,20161009,0.0,0.0,2.79,102.6,118.52,主要诊断,G61.801,慢性炎症性脱髓鞘性多发性神经病


In [36]:
# 显示可能为 颅内感染的病例
csf_cd[csf_cd.ICD.str.contains('A')]

,IPD_ID,check_time,csf_wbc,csf_rbc,csf_glu,csf_tp,csf_cl,diag_type,ICD,diagnosis
1,ZY010000061218,20160917,NaN,NaN,6.70,47.3,111.93,主要诊断,A86.x00,病毒性脑炎
26,ZY020000052413,20170421,NaN,NaN,3.84,46.3,123.27,其他诊断,A53.900,梅毒
39,ZY020000105143,20180524,NaN,NaN,1.35,128.8,102.90,其他诊断,A16.010,肺结核
40,ZY020000105143,20180524,NaN,NaN,1.35,128.8,102.90,其他诊断,A17.000+,结核性脑膜炎可能性大
51,ZY020000105143,20180528,NaN,NaN,1.56,139.0,110.00,其他诊断,A16.010,肺结核
52,ZY020000105143,20180528,NaN,NaN,1.56,139.0,110.00,其他诊断,A17.000+,结核性脑膜炎可能性大
63,ZY020000105143,20180526,NaN,NaN,2.14,153.7,106.80,其他诊断,A16.010,肺结核
64,ZY020000105143,20180526,NaN,NaN,2.14,153.7,106.80,其他诊断,A17.000+,结核性脑膜炎可能性大
77,ZY010000068779,20170103,NaN,NaN,3.12,53.5,122.70,其他诊断,A17.804+G05.0*,结核性病待排
92,ZY010000043034,20151211,39.0,2.0,2.62,22.2,118.67,主要诊断,A87.900,病毒性脑膜炎


In [37]:
# drop tp值过大的observations(正常值 15-45g/dl，>3g)
csf_cd = csf_cd[csf_cd.csf_tp < 300]

# 我们最终有了两个表
    1. dis_he：用于health economic分析
    2. csf_cd：用于csf检验结果与诊断关系分析

In [ ]:
csf_cd.info()

# 3. 数据描述 + plotting
    3.1 单一数据
        3.1.1 均值 mean
        3.1.2 方差 variance
        3.1.3 标准差 standard deviation
    3.2 多个数据

### 3.1.1 mean
    离散变量平均数的数学公式: $$\mu=\frac{\sum\limits_{i=1}^{n}x_i}{N}$$

$$\mu=\frac{\sum\limits_{i=1}^{n}x_i}{N}$$

In [ ]:
data = csf_cd[csf_cd['csf_tp'].isna()==False][['csf_tp']]
fig, ax = plt.subplots(figsize = (15, 6))
_ = sns.distplot(data)

In [ ]:
csf_cd['csf_tp'].mean()

### 3.2 variance
    方差的数学公式(population) $$ \sigma^2=\frac{1}{N}\sum\limits_{i=1}^{n} (x_i-\mu)^2  $$

$$ \sigma^2=\frac{1}{N}\sum\limits_{i=1}^{n} (x_i-\mu)^2  $$

    方差的数学公式(sample)

$$ S^2=\frac{1}{n-1}\sum\limits_{i=1}^{n} (x_i-\bar{x})^2  $$

In [ ]:
csf_cd['csf_tp'].var()

### 3.3 standard deviation
    标准差的数学公式： $$ \sigma = \sqrt{variance} \mspace{6mu} or \mspace{6mu} \sqrt{\sigma^2} $$

$$ \sigma = \sqrt{variance} \mspace{6mu} or \mspace{6mu} \sqrt{\sigma^2} $$ 

In [ ]:
csf_cd['csf_tp'].std()

In [ ]:
csf_cd.describe()

## plot two or several variables/columns

    1.scatter(散点图): two continous数据
    2.box or bar: one continous by one categorical/dummy 数据
    3.two categorical/dummy: cross table: 第四部分有示例

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
_ = sns.scatterplot('nondrug_fee', 'drug_fee', data=dis_he)
ax.set_title('nondrug/drug of discharged patients')

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
_ = sns.boxplot(y = 'stay_days', x = 'adm_site', data=dis_he)

# 4. 统计分析 + 作图

    1.t-test
    2.ANOVA
    3.regression: OLS 
    4.nonparametric testing（非参数检验）：略
    5.Kaplan Meier plot using lifelines module：略

### 4.1 t-test by scipy.stat

#### T-test(两样本) t值的计算

$$ t = \frac{\bar{X_1}-\bar{X_2}}{\sqrt{\frac{S_1^2}{N_1}+\frac{S_2^2}{N_2}}}$$

#### 按照自由度df=N1+N2-1，计算/查表获取 p value

In [ ]:
# 比较自两个正态分布中取得的数值，是否存在统计学差异

In [ ]:
data_ttest_a = np.random.normal(0,10,10000)
data_ttest_b = np.random.normal(0,10,10000)

In [ ]:
fig_ttest = plt.figure(figsize=(10,6))
ax_ttest = fig_ttest.add_subplot(111)
sns.distplot(data_ttest_a)
sns.distplot(data_ttest_b)

In [ ]:
stats.ttest_ind(data_ttest_a, data_ttest_b)

### 4.2 ananlysis of variance（ANOVA） 方差分析
    实际数据，dis_he: 不同天（星期几）入院对住院日的影响

In [ ]:
# 生成variable：周几数据
dis_he['weekday']=dis_he.adm_DT.dt.weekday_name

In [ ]:
dis_he.groupby(['weekday'])['stay_days'].mean()

In [ ]:
# 作图，sns的barplot
week_list = ['Monday', 'Tuesday', 'Thursday', 'Wednesday', 'Friday', 'Saturday', 'Sunday']
fig = plt.figure(figsize=[10, 9])
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)
sns.set(style='ticks')
sns.barplot(x='weekday', y='stay_days',  data=dis_he, ax=ax1,
            order=week_list)
sns.barplot(x='weekday', y='stay_days',  data=dis_he, ax=ax2, 
            hue='adm_site', order=week_list)

### ANOVA 方差分析：不同天入院住院日是否存在差别
    ANOVA的假设用公式表示是

$H_0:\mu_1=\mu_2=\mu_3=\mu_4=\mu_5=\mu_6=\mu_7 \\ H_\alpha:\mu_i\ne\mu_j$

In [ ]:
from statsmodels.formula.api import ols

In [ ]:
stay_weekday_lm = ols('stay_days ~ C(weekday, Sum)', data=dis_he).fit()

In [ ]:
table = sm.stats.anova_lm(stay_weekday_lm, typ=2)

In [ ]:
print(table)

In [ ]:
## 4.3 回归，线性 

In [ ]:
sns.lmplot(x='nondrug_fee', y='drug_fee', data=dis_he, height=8)

In [ ]:
# OLS回归模型
ols_drug_fee = smf.ols('drug_fee ~ nondrug_fee', data=dis_he).fit()

In [ ]:
print(ols_drug_fee.summary())

In [ ]:
ols_drug_fee_stay = smf.ols('drug_fee ~ nondrug_fee + stay_days', data=dis_he).fit()
print(ols_drug_fee_stay.summary())

In [ ]:
sns.lmplot(x='stay_days', y='drug_fee', data=dis_he, height=8)

# 5. bonus: 机器学习（machine learning）or 人工智能（AI）
    有限情况下的示例：神经内科按年度，月度来看，入院患者数量有没有规律。
    Note：传统的统计分析是提供结论的，不能提供深度研究的价值

In [ ]:
# 提取年度信息
dis_he['year']=dis_he.adm_DT.dt.year

In [ ]:
# 提取月度信息
dis_he['month_name']=dis_he.adm_DT.dt.month_name()

In [ ]:
# 重新简化一个表格
dis_heat = dis_he[['IPD_ID', 'year', 'month_name']]

In [ ]:
df = pd.crosstab(dis_heat['year'], dis_heat['month_name'])

In [ ]:
df.drop(index = [2013, 2019], inplace=True)

In [ ]:
# plot clustermap
ax = sns.clustermap(df, cmap='magma_r')  # annot=True

# 后记
    1.临床还没有真正意义的“big data”，好的研究设计，适量的数据。
        尤甚：数据淹没个性，但突出个性更需要大数据。
             如何做到：通过network放大个性
             
    2.市面上的付费软件，容易上手，但深度达不到（除R）
    3.我的目的，以及后续想法
        3.1 学习：启发几位高手，共同进步。
            note: 10,000小时定律，不用记忆具体code
        3.2 使用：诸位了解什么是可以做的，可以交给我来做（但你必须要懂，也要完成很多工作）